In [1]:
import pandas as pd

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [3]:
!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria

Dataset URL: https://www.kaggle.com/datasets/iarunava/cell-images-for-detecting-malaria
License(s): unknown
 98% 665M/675M [00:06<00:00, 173MB/s]
100% 675M/675M [00:06<00:00, 106MB/s]


In [5]:
import zipfile
zip_ref = zipfile.ZipFile('/content/cell_image.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [6]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Flatten
from keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [23]:
conv_base = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(150, 150, 3)
)

In [24]:
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [25]:
conv_base.trainable = False

In [26]:
data = keras.utils.image_dataset_from_directory(
    directory = '/content/cell_images',
    labels='inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size=(150, 150)
)

Found 27558 files belonging to 2 classes.


In [27]:
# Calculate the number of batches for train and test
total_batches = tensorflow.data.experimental.cardinality(data).numpy()
train_size = int(0.8 * total_batches)

In [28]:
train_ds = data.take(train_size)
test_ds = data.skip(train_size)

In [29]:
def process(image, label):
  image = tensorflow.cast(image/255., tensorflow.float32)
  return (image, label)

train_ds = train_ds.map(process)
test_ds =  test_ds.map(process)

In [30]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [32]:
history = model.fit(train_ds, epochs=10, validation_data=test_ds)

Epoch 1/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 92s 133ms/step - accuracy: 0.9456 - loss: 0.1516 - val_accuracy: 0.9461 - val_loss: 0.1557
Epoch 2/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 77s 112ms/step - accuracy: 0.9503 - loss: 0.1362 - val_accuracy: 0.9443 - val_loss: 0.1455
Epoch 3/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 77s 112ms/step - accuracy: 0.9510 - loss: 0.1332 - val_accuracy: 0.9414 - val_loss: 0.1521
Epoch 4/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 77s 112ms/step - accuracy: 0.9598 - loss: 0.1154 - val_accuracy: 0.9437 - val_loss: 0.1512
Epoch 5/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 77s 112ms/step - accuracy: 0.9582 - loss: 0.1168 - val_accuracy: 0.9456 - val_loss: 0.1456
Epoch 6/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 82s 112ms/step - accuracy: 0.9617 - loss: 0.1072 - val_accuracy: 0.9436 - val_loss: 0.1510
Epoch 7/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 92s 133ms/step - accuracy: 0.9660 - loss: 0.0938 - val_accuracy: 0.9481 - val_loss: 0.1466
Epoch 8/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 142s 133ms/step - accuracy: 0.9707 - loss: 